In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bipolar_disorder"
cohort = "GSE92538"

# Input paths
in_trait_dir = "../../input/GEO/Bipolar_disorder"
in_cohort_dir = "../../input/GEO/Bipolar_disorder/GSE92538"

# Output paths
out_data_file = "../../output/preprocess/Bipolar_disorder/GSE92538.csv"
out_gene_data_file = "../../output/preprocess/Bipolar_disorder/gene_data/GSE92538.csv"
out_clinical_data_file = "../../output/preprocess/Bipolar_disorder/clinical_data/GSE92538.csv"
json_path = "../../output/preprocess/Bipolar_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
As a domain expert in this biomedical research project, I'll provide the corrected code for this step:

```python
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Check for gene expression data availability
# Based on the background information, this dataset contains transcriptomic data
# from human brain (dorsolateral prefrontal cortex) with 11,911 ENTREZ transcripts
is_gene_available = True  # Gene expression data is available

# 2. Identify variable availability and create conversion functions

# 2.1 Trait (Bipolar disorder)
# From the sample characteristics, trait information is in row 2 ("diagnosis: ...")
trait_row = 2

# 2.2 Age information
# Age is available in row 8
age_row = 8

# 2.3 Gender information
# Gender is available in row 6
gender_row = 6

# Conversion functions
def convert_trait(value):
    """Convert diagnosis information to binary trait indicator for Bipolar disorder."""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if it's Bipolar Disorder (case-insensitive)
    if 'bipolar' in value.lower():
        return 1
    elif value.lower() in ['control', 'major depressive disorder', 'schizophrenia']:
        return 0
    else:
        return None  # Unknown or other diagnosis

def convert_age(value):
    """Convert age information to continuous numerical value."""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender information to binary (0=female, 1=male)."""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary values
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None  # Unknown gender

# 3. Save metadata through initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if available
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    sample_chars = {
        0: ['cohort: Schiz Cohort 1', 'cohort: Dep Cohort 3', 'cohort: Dep Cohort 4', 'cohort: Dep Cohort 2', 'cohort: Dep Cohort 1'], 
        1: ['site of processing: UC_Davis', 'site of processing: UC_Irvine', 'site of processing: U_Michigan'], 
        2: ['diagnosis: Schizophrenia', 'diagnosis: Control', 'diagnosis: Major Depressive Disorder', 'diagnosis: Bipolar Disorder'], 
        3: ['subject id: 101078', 'subject id: 101244', 'subject id: 101283', 'subject id: 101309', 'subject id: 101431', 'subject id: 101488', 'subject id: 101597', 'subject id: 101657', 'subject id: 101674', 'subject id: 101690', 'subject id: 101736', 'subject id: 101739', 'subject id: 101811', 'subject id: 101923', 'subject id: 101956', 'subject id: 102038', 'subject id: 102118', 'subject id: 102212', 'subject id: 102420', 'subject id: 102422', 'subject id: 102432', 'subject id: 102440', 'subject id: 102475', 'subject id: 102513', 'subject id: 102539', 'subject id: 102636', 'subject id: 102785', 'subject id: 103037', 'subject id: 103068', 'subject id: 103091'], 
        4: ['agonal factor: 0', 'agonal factor: NA', 'agonal factor: 1', 'agonal factor: 2', 'agonal factor: 3'], 
        5: ['tissue ph (cerebellum): 6.83', 'tissue ph (cerebellum): 6.97', 'tissue ph (cerebellum): 7.01', 'tissue ph (cerebellum): NA', 'tissue ph (cerebellum): 6.87', 'tissue ph (cerebellum): 7.05', 'tissue ph (cerebellum): 6.38', 'tissue ph (cerebellum): 6.72', 'tissue ph (cerebellum): 6.91', 'tissue ph (cerebellum): 6.06', 'tissue ph (cerebellum): 7', 'tissue ph (cerebellum): 7.02', 'tissue ph (cerebellum): 6.86', 'tissue ph (cerebellum): 6.54', 'tissue ph (cerebellum): 7.21', 'tissue ph (cerebellum): 6.63', 'tissue ph (cerebellum): 6.42', 'tissue ph (cerebellum): 6.89', 'tissue ph (cerebellum): 7.19', 'tissue ph (cerebellum): 6.68', 'tissue ph (cerebellum): 6.62', 'tissue ph (cerebellum): 7.17', 'tissue ph (cerebellum): 6.84', 'tissue ph (cerebellum): 6.76', 'tissue ph (cerebellum): 6.93', 'tissue ph (cerebellum): 6.59', 'tissue ph (cerebellum): 6.61', 'tissue ph (cerebellum): 6.79', 'tissue ph (cerebellum): 7.2', 'tissue ph (cerebellum): 6.81'], 
        6: ['gender: M', 'gender: F'], 
        7: ['race: Caucasian', 'race: Hispanic', 'race: Asian', 'race: Other', 'race: African American'], 
        8: ['age: 39', 'age: 32', 'age: 63', 'age: 70', 'age: 71', 'age: 44', 'age: 66', 'age: 59', 'age: 65', 'age: 69', 'age: 52', 'age: 49', 'age: 58', 'age: 45', 'age: 72', 'age: 73', 'age: 64', 'age: 77', 'age: 55', 'age: 53', 'age: 62', 'age: 47', 'age: 67', 'age: 57', 'age: 35', 'age: 48', 'age: 19', 'age: 54', 'age: 50', 'age: 56'], 
        9: ['post-mortem interval: 7.5', 'post-mortem interval: 27.25', 'post-mortem interval: 29.5', 'post-mortem interval: 14.5', 'post-mortem interval: 36.5', 'post-mortem interval: 23', 'post-mortem interval: 32', 'post-mortem interval: 13.25', 'post-mortem interval: 26', 'post-mortem interval: 11.25', 'post-mortem interval:


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I understand the issue now. I need to extract and parse the GEO files, focusing on the .soft file which contains the clinical information.

```python
import pandas as pd
import numpy as np
import os
import json
import gzip
import re
from typing import Callable, Optional, Dict, Any

# Function to extract and parse the .soft file
def parse_geo_soft_file(file_path):
    # Dictionary to store sample characteristics by sample_id
    sample_characteristics = {}
    current_sample = None
    
    # Open the gzipped file and decode it
    with gzip.open(file_path, 'rt') as f:
        for line in f:
            line = line.strip()
            
            # Identify when we start a new sample
            if line.startswith("^SAMPLE = "):
                current_sample = line.split("^SAMPLE = ")[1]
                sample_characteristics[current_sample] = {}
            
            # Extract sample characteristics
            elif current_sample and line.startswith("!Sample_characteristics_ch1 = "):
                char_value = line.replace("!Sample_characteristics_ch1 = ", "").strip()
                if ":" in char_value:
                    key, value = char_value.split(":", 1)
                    sample_characteristics[current_sample][key.strip()] = value.strip()
                else:
                    # Handle characteristics without a key-value format
                    sample_characteristics[current_sample][char_value] = True
            
            # Extract sample title
            elif current_sample and line.startswith("!Sample_title = "):
                title = line.replace("!Sample_title = ", "").strip()
                sample_characteristics[current_sample]["title"] = title
    
    return sample_characteristics

# First, check if the soft file exists
soft_file_path = os.path.join(in_cohort_dir, "GSE92538_family.soft.gz")
if os.path.exists(soft_file_path):
    print(f"Parsing SOFT file: {soft_file_path}")
    sample_data = parse_geo_soft_file(soft_file_path)
    
    # Convert to DataFrame for easier analysis
    samples_list = []
    for sample_id, characteristics in sample_data.items():
        sample_dict = {'sample_id': sample_id}
        sample_dict.update(characteristics)
        samples_list.append(sample_dict)
    
    if samples_list:
        clinical_df = pd.DataFrame(samples_list)
        print("Clinical data structure:")
        print(clinical_df.head())
        print(f"Number of samples: {len(clinical_df)}")
        print(f"Columns: {clinical_df.columns.tolist()}")
        
        # Check unique values in relevant columns to identify trait, age, and gender
        for col in clinical_df.columns:
            if col != 'sample_id':
                unique_vals = clinical_df[col].unique()
                print(f"Column '{col}' unique values: {unique_vals}")
                
                # Look for columns containing relevant information
                col_lower = col.lower()
                if 'disease' in col_lower or 'diagnosis' in col_lower or 'status' in col_lower or 'group' in col_lower:
                    print(f"Potential trait column: {col}")
                elif 'age' in col_lower:
                    print(f"Potential age column: {col}")
                elif 'gender' in col_lower or 'sex' in col_lower:
                    print(f"Potential gender column: {col}")
        
        # Check if gene expression data is available by looking for matrix files
        is_gene_available = any(f.endswith('_series_matrix.txt.gz') for f in os.listdir(in_cohort_dir))
        
        # Now, based on the column analysis, define trait_row, age_row, and gender_row
        # These will be set based on the analysis output
        trait_row = None
        age_row = None
        gender_row = None
        
        # For demonstration, let's create a transposed clinical data for geo_select_clinical_features
        # In a real scenario, we would identify the specific rows from the DataFrame
        data_dict = {}
        for i, col in enumerate(clinical_df.columns):
            if col != 'sample_id':
                data_dict[i] = clinical_df[col].tolist()
        
        transposed_clinical_df = pd.DataFrame(data_dict, index=clinical_df['sample_id'])
        
        # Check the title column for trait information
        if 'title' in clinical_df.columns:
            title_unique = clinical_df['title'].unique()
            print(f"Sample titles: {title_unique}")
            
            # Check if titles contain bipolar/control information
            has_bipolar_info = any('bipolar' in str(title).lower() or 'control' in str(title).lower() for title in title_unique)
            if has_bipolar_info:
                trait_row = list(clinical_df.columns).index('title') - 1  # Adjust for sample_id column
                print(f"Found trait information in 'title' column, row: {trait_row}")
        
        # Conversion functions based on the data analysis
        def convert_trait(value):
            if pd.isna(value):
                return None
            value_str = str(value).lower()
            
            if 'control' in value_str or 'healthy' in value_str or 'normal' in value_str:
                return 0
            elif 'bipolar' in value_str or 'bd' in value_str or 'bp' in value_str or 'patient' in value_str:
                return 1
            return None
        
        def convert_age(value):
            if pd.isna(value):
                return None
            try:
                # Extract numbers from text
                nums = re.findall(r'\d+\.?\d*', str(value))
                if nums:
                    return float(nums[0])
                return None
            except:
                return None
        
        def convert_gender(value):
            if pd.isna(value):
                return None
            value_str = str(value).lower()
            
            if 'female' in value_str or 'f' == value_str:
                return 0
            elif 'male' in value_str or 'm' == value_str:
                return 1
            return None
        
        # For this dataset, determine trait availability based on analysis
        is_trait_available = trait_row is not None
        
        # Save metadata
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )
        
        # Extract clinical features if trait is available
        if is_trait_available:
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=transposed_clinical_df,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age if age_row is not None else None,
                gender_row=gender_row,
                convert_gender=convert_gender if gender_row is not None else None
            )
            
            # Preview the selected clinical features
            print("Preview of selected clinical features:")
            preview = preview_df(selected_clinical_df)
            print(preview)
            
            # Save the selected clinical features to a CSV file
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        else:
            print("Clinical feature extraction skipped: Trait data not available")
    else:
        print("No sample data found in the SOFT file")
        is_gene_available = any(f.endswith('_series_matrix.txt.gz') for f in os.listdir(in_cohort_dir))
        validate_and_save_cohort_info(
            is_final=False,


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# Examining the gene identifiers
# The format appears to be Affymetrix probe IDs (format: XXXXX_at)
# These are not standard human gene symbols and need to be mapped
# Affymetrix probe IDs typically need to be mapped to gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check if there are any columns that might contain gene information
sample_row = gene_annotation.iloc[0].to_dict()
print("\nFirst row as dictionary:")
for col, value in sample_row.items():
    print(f"{col}: {value}")

# Check if the SOFT file has the right information for gene mapping
print("\nFurther examination needed - this might be a miRNA dataset or using non-standard annotations")
print("Looking at the index of gene_data to compare with annotation ID format:")
print(gene_data.index[:5])
print("\nComparing to annotation ID format:")
print(gene_annotation['ID'].head())

# Check if there's a mismatch between gene data IDs and annotation IDs
id_match = any(gene_data.index[0] in str(x) for x in gene_annotation['ID'])
print(f"\nDirect ID match between gene data and annotation: {id_match}")

# Since we identified this as requiring gene mapping but suitable annotation isn't found in this file,
# let's examine if this is a complex series with multiple platforms
print("\nThis appears to be a GSE with multiple platforms or a SuperSeries.")
print("The background information indicated: 'This SuperSeries is composed of the SubSeries listed below.'")
print("The current annotation file may not correspond to the gene expression matrix.")


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Determine which columns in gene_annotation contain probe IDs and gene symbols
# From the previous output, we can see:
# - 'ID' column in gene_annotation contains identifiers like "10000_at" which match gene_data index
# - 'SYMBOL' column contains gene symbols like "AKT3"

# 2. Get a gene mapping dataframe by extracting the two relevant columns
mapping_df = get_gene_mapping(gene_annotation, prob_col="ID", gene_col="SYMBOL")

print("Gene mapping dataframe preview:")
print(mapping_df.head())
print(f"Shape of mapping dataframe: {mapping_df.shape}")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("After mapping, gene expression data preview:")
print(f"Shape of gene expression data: {gene_data.shape}")
print(f"First 10 gene symbols: {gene_data.index[:10].tolist()}")

# Normalize gene symbols in the index
gene_data = normalize_gene_symbols_in_index(gene_data)

print("After normalizing gene symbols:")
print(f"Shape of normalized gene expression data: {gene_data.shape}")
print(f"First 10 normalized gene symbols: {gene_data.index[:10].tolist()}")

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. We already normalized gene symbols in the gene expression data in step 7
# Load the normalized gene data that was saved in the previous step
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Loaded gene expression data with shape: {gene_data.shape}")

# Reload the background and clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 2. Link the clinical and genetic data
# Define conversion functions:
def convert_trait(value):
    """Convert bipolar disorder status to binary format."""
    if not isinstance(value, str):
        return None
    value = value.lower()
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
        
    if "bipolar disorder" in value:
        return 1  # Has bipolar disorder
    elif value in ["control", "schizophrenia", "major depressive disorder"]:
        return 0  # Control/other diagnosis
    return None

def convert_age(value):
    """Convert age data to continuous format."""
    if not isinstance(value, str):
        return None
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if value == "f":
        return 0
    elif value == "m":
        return 1
    return None

# Get clinical data using the correct row indices
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=2,     # Based on output showing diagnosis in row 2
    convert_trait=convert_trait,
    age_row=8,       # Age data is in row 8 based on first step output
    convert_age=convert_age,
    gender_row=6,    # Gender data is in row 6 based on first step output
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from brain samples with diagnoses including Bipolar Disorder, Major Depressive Disorder, Schizophrenia, and Control."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")